![FUNCIONES](src/PORT6.jpg)

# **Funciones de la API**

*A continuación se realiza la definición de las funciones que tendrá la API.*

**Nota:** *Estas funciones se crearán en el archivo [main](main.py)*

#### **Importación de librerías** ####
---

*Para este cuaderno usaremos las siguientes librerías: **pandas** y **pyarrow**.*

In [1]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
dfUsReviews= pd.read_parquet("dataout/out_users_reviews.parquet")
dfUsers= pd.read_parquet("dataout/out_users.parquet")
dfUsersItems= pd.read_parquet("dataout/out_users_items.parquet")
dfGames= pd.read_parquet("dataout/out_games.parquet")
dfGenres = pd.read_parquet("dataout/out_genres_games.parquet")
dfFeelings = pd.read_parquet("dataout/out_feelings.parquet")

#### **Función developer** ####
---

*La función **developer** le permite al usuario **filtrar** por un **desarrollador** específico y mostrarle la **cantidad** de apps lanzadas por **año** y el porcentaje de aplicaciones que fueron **gratis** para ese año.*

#### Creamos el dataframe para facilitar esta función ####

*Este dataframe solo contará con las columnas que nos interesan:*

1. *IdApp.*
2. *Developer.*
3. *Price.*
4. *ReleaseYear.*


In [7]:
dfDeveloper= pd.read_parquet("datasets/out_games.parquet")
dfDeveloper = dfDeveloper[["IdApp","Developer","Price","ReleaseYear"]]
dfDeveloper["Developer"] = dfDeveloper["Developer"].str.upper()
dfDeveloper.head(1)

,IdApp,Developer,Price,ReleaseYear
0,10,VALVE,9.99,2000


#### Creamos la función developer ####

*Esta función recibirá como parametro:*

1. *dev = Cadena string con el nombre del desarrollador.*


In [8]:
def developer(dev:str, data = dfDeveloper):
    # Formateamos el string de entrada para un correcto filtrado
    dev = dev.upper()
    
    #Filtramos el dataframe y solo nos quedamos con el developer que queremos.
    dfDev = data[data["Developer"] == dev]
    
    #Creamos la agrupación por año de lanzamiento y calculamos la cantidad de aplicaciones de dicho año
    countPerYear = dfDev.groupby("ReleaseYear").size().reset_index(name="Cantidad de Apps")
    
    #Creamos un dataframe con los años de lanzamiento que tienen aplicaciones Free == 0
    priceFreePerYear = dfDev[dfDev["Price"] == 0]
    priceFreePerYear = priceFreePerYear.groupby("ReleaseYear").size().reset_index(name="PriceFreeCount")
    
    #Creamos un inner join que une countperYear con PriceFreeYear en la columna release year
    countPerYear = pd.merge(countPerYear, priceFreePerYear, on="ReleaseYear", how="left")
    
    # Creamos la tabla contenido free que es la cantidad de apps con precio gratis sobre el total de aplicacciones.
    countPerYear["Contenido Free"] = round((countPerYear["PriceFreeCount"] / countPerYear["Cantidad de Apps"]) * 100, 2)
    
    # Llenamos los valores NAN resultado del inner join y los ponemos en 0.
    countPerYear = countPerYear.fillna(0)
    
    dfDev = countPerYear[["ReleaseYear", "Cantidad de Apps", "Contenido Free"]].rename(columns={"ReleaseYear":"Año"})
    
    return dfDev

*Revisamos que funcione correctamente:*

In [9]:
developer("gearbox Software")

,Año,Cantidad de Apps,Contenido Free
0,1999,1,0.0
1,2001,1,0.0
2,2005,2,0.0
3,2008,1,0.0
4,2009,2,0.0
5,2010,3,0.0
6,2011,2,0.0
7,2013,17,0.0
8,2016,2,50.0


In [10]:
developer("blue byte")

,Año,Cantidad de Apps,Contenido Free
0,2008,1,0.0
1,2010,4,25.0
2,2011,2,0.0
3,2013,1,100.0
4,2015,2,50.0
5,2016,1,0.0


#### Guardamos el dataframe para las consultas de la API ####

*Lo exportamos a parquet:*

In [11]:
devTab = pa.Table.from_pandas(dfDeveloper)
dir = "datafunc/developer.parquet"
pq.write_table(devTab,dir)

#### **Función userdata** ####
---

*La función **userData** le permite al usuario **filtrar** por el **id de usuario** específico y mostrarle la **cantidad de dinero gastado**, el porcentaje de **recomedación** y la **cantidad de items** que adquirió.*

#### Creamos el dataframe para facilitar esta función ####

*Este dataframe solo contará con las columnas que nos interesan:*

1. *IdUserSteam.*
2. *IdApp.*
2. *IsRecommend.*
4. *Price.*

*Unimos el dataframe **UsersItems** con el dataframe **dfGames**, en la columna IdApp*

1. *Con el fin de traer la columna Price alojada en **dfGames**.*
2. *Nos quedamos solo con las columnas que necesitaremos.*

In [12]:
dfUserData = pd.merge(dfUsersItems, dfGames, on="IdApp", how="left")
dfUserData = dfUserData[["IdUserSteam","IdApp","Price"]]

*Ahora necesitamos transformar el dataframe **dfUsReviews** para que nos muestre el porcentaje de **recomendación** por aplicación.*

1. *Agrupamos por **IdApp** y contamos la cantidad de **Reviews** que tiene ese Id*
2. *Contamos la cantidad de veces que **True** aparece en la columna.*
3. *Unimos los resultados con el conteo.*


In [13]:
# Paso 1
dfRecommends = dfUsReviews.groupby('IdApp').size().reset_index(name='TotalReviews')

# Paso 2
conteo_recomendado = dfUsReviews[dfUsReviews['IsRecommend']].groupby('IdApp').size().reset_index(name='TrueCount')

# Paso 3:
dfRecommends = pd.merge(dfRecommends, conteo_recomendado, on='IdApp', how='left')
dfRecommends['RecommendsPercen'] = round((dfRecommends['TrueCount'] / dfRecommends['TotalReviews']) * 100,2)
dfRecommends = dfRecommends.fillna(0)

# Mostrar el DataFrame resultante
dfRecommends.head(5)


,IdApp,TotalReviews,TrueCount,RecommendsPercen
0,10,57,56.0,98.25
1,20,17,11.0,64.71
2,30,4,3.0,75.00
3,40,1,1.0,100.00
4,50,4,3.0,75.00


*Unimos el dataframe **dfUserData** con el dataframe **dfRecommends**, en la columna IdApp.*

1. *Con el fin de traer la columna **RecommendsPercen**.*
2. *Nos quedamos solo con las columnas que necesitaremos.*

In [14]:
dfUserData = pd.merge(dfUserData, dfRecommends, on="IdApp", how="left")
dfUserData = dfUserData[["IdUserSteam","IdApp","Price","RecommendsPercen"]]

*Unimos el dataframe **dfUserData** con el dataframe **dfUsers**, en la columna IdUserSteam*

1. *Con el fin de traer la columna **PurchasedItems**.*
2. *Nos quedamos solo con las columnas que necesitaremos.*

In [15]:
dfUserData = pd.merge(dfUserData, dfUsers, on="IdUserSteam", how="left")
dfUserData = dfUserData[["IdUserSteam","IdApp","Price","RecommendsPercen","PurchasedItems"]]

*Revisamos el resultado de las consultas.*

In [16]:
dfUserData.head(5)

,IdUserSteam,IdApp,Price,RecommendsPercen,PurchasedItems
0,76561197960269200,306660,14.99,100.00,384
1,76561197960269200,12150,9.99,0.00,384
2,76561197960269200,4700,19.99,95.65,384
3,76561197960269200,4760,9.99,100.00,384
4,76561197960269200,10500,19.99,100.00,384


#### Creamos la función userdata ####

*Esta función recibirá como parametro:*

1. *idUser = Cadena string con el número de id del usuario que se quiere consultar.*

In [17]:
def userdata(idUser:str, data=dfUserData):
    # Formateamos el string de entrada a número.
    idUser = int(idUser)
        
    #Filtramos el dataframe y solo nos quedamos con el idusuario que queremos.
    dfUs = data[data["IdUserSteam"] == idUser]
    
    #Agrupamos por el usuario indicado.
    groupUs = dfUs.groupby("IdUserSteam")
    
    #Calculamos la suma de los precios de los items que adquirió el usuario.
    totPrice = groupUs["Price"].sum().reset_index(name="Dinero Gastado")
    
    #Calculamos el promedio de recomendación de las apps que el usuario compró.
    avgRecom = groupUs["RecommendsPercen"].mean().reset_index(name="Porcentaje de Recomendación")
    
    # Contamos la cantidad de items que compró el usuario.
    purchasedItems = groupUs.size().reset_index(name='Cantidad de items')

    # Unimos los resultados de las consultas.
    dfUs = pd.merge(totPrice, avgRecom, on='IdUserSteam')
    dfUs = pd.merge(dfUs, purchasedItems, on='IdUserSteam')
    
    return dfUs

*Revisamos que funcione correctamente:*

In [18]:
userdata("76561197960269200")

,IdUserSteam,Dinero Gastado,Porcentaje de Recomendación,Cantidad de items
0,76561197960269200,4598.53,83.513048,384


In [19]:
userdata("76561198329548331")

,IdUserSteam,Dinero Gastado,Porcentaje de Recomendación,Cantidad de items
0,76561198329548331,0.0,52.305,7


#### Guardamos el dataframe para las consultas de la API ####

*Lo exportamos a parquet*.

In [20]:
userTab = pa.Table.from_pandas(dfUserData)
dir1 = "datafunc/userdata.parquet"
pq.write_table(userTab,dir1)

#### **Función userForGenre** ####
---

*La función **userForGenre** le permite al usuario **filtrar** por el **genero** específico de la aplicación y mostrarle el **usuario con más horas jugadas**, y la **lista de horas acumuladas por año de lanzamiento**.*

#### Creamos el dataframe para facilitar esta función ####

*Este dataframe solo contará con las columnas que nos interesan:*

1. *IdUserSteam*
2. *IdApp*
3. *Playtime*
4. *ReleaseYear*
5. *Genres*

*Unimos el dataframe **dfUsersItems** con el dataframe **dfGenres**, en la columna IdApp*

1. *Con el fin de traer las columnas de los **géneros**.*
2. *Nos quedamos solo con las columnas que necesitaremos.*

In [35]:
dfUxG = pd.merge(dfUsersItems,dfGenres, on="IdApp", how="left")
dfUxG.drop(columns=["NameApp","PlaytimeLast2W"], inplace=True)
dfUxG.columns = dfUxG.columns.str.upper()
dfUxG.rename(columns={"IDAPP": "IdApp", "IDUSERSTEAM": "IdUserSteam", "PLAYTIME": "PlayTime"}, inplace=True)
dfUxG.head(1)

,IdUserSteam,IdApp,PlayTime,ACTION,ADVENTURE,ANIMATION & MODELING,AUDIO PRODUCTION,CASUAL,DESIGN & ILLUSTRATION,EARLY ACCESS,...,PHOTO EDITING,RPG,RACING,SIMULATION,SOFTWARE TRAINING,SPORTS,STRATEGY,UTILITIES,VIDEO PRODUCTION,WEB PUBLISHING
0,76561197960269200,306660,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


*Unimos el dataframe **dfUxG** con el dataframe **dfGames**, en la columna IdApp*

1. *Con el fin de traer la columna **ReleaseYear**.*
2. *Nos quedamos solo con las columnas que necesitaremos.*

In [36]:
dfUxG = pd.merge(dfUxG, dfGames, on="IdApp", how="left")
dfUxG.drop(columns=["Name", "Developer", "Publisher", "Price", "ReleaseMonth"], inplace=True)
dfUxG.head(1)

,IdUserSteam,IdApp,PlayTime,ACTION,ADVENTURE,ANIMATION & MODELING,AUDIO PRODUCTION,CASUAL,DESIGN & ILLUSTRATION,EARLY ACCESS,...,RPG,RACING,SIMULATION,SOFTWARE TRAINING,SPORTS,STRATEGY,UTILITIES,VIDEO PRODUCTION,WEB PUBLISHING,ReleaseYear
0,76561197960269200,306660,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,2014


#### Creamos la función userForGenre ####

*Esta función recibirá como parametros: **genre** y el **dataframe**:*

1. *genre = Cadena string con el nombre del genro que se quiere consultar.*

In [41]:
def userforGenre(genre:str, data = dfUxG):
    # Formateamos el string de entrada y lo capitalizamos.
    genre = genre.upper()

    #Filtramos el dataframe y solo nos quedamos con el genero que queremos y las filas que solo correspondan a ese genero.
    data = data[["IdUserSteam","PlayTime","ReleaseYear", genre]]
    data = data[data[genre]==1]
    data.drop(columns=genre, inplace=True)
    
    #Agrupamos por usuario y calculamos la suma de horas jugadas por cada usuario.
    groupByUs = data.groupby('IdUserSteam')['PlayTime'].sum().reset_index()
    
    #Encontramos el usuario con mayor número de horas de playtime usando la funcion idxmax() que permite encontrar el valor max.
    userMax = groupByUs.loc[groupByUs['PlayTime'].idxmax()]
    
    #Creamos las columnas por año y calculamos las sumas por año de ese usuario. Usamos pivot para convertir los valores en columnas
    dfMax = data[data['IdUserSteam'] == userMax['IdUserSteam']]
    dfPivoteado = dfMax.pivot_table(index='IdUserSteam', columns='ReleaseYear', values='PlayTime', aggfunc='sum').fillna(0)
    
    # Mostrar el resultado
    print("IdUserSteam con más horas de Playtime:", userMax['IdUserSteam'])
    print("Total de horas de Playtime del usuario:", userMax['PlayTime'])   
    return dfPivoteado

*Revisamos que funcione correctamente:*

In [42]:
userforGenre("Casual")

IdUserSteam con más horas de Playtime: 76561197995207074
Total de horas de Playtime del usuario: 1221941


ReleaseYear,1999,2002,2004,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
IdUserSteam,,,,,,,,,,,,,,
76561197995207074,0,0,6985,0,1241,2870,12066,78658,29722,107965,121915,362201,498260,58


In [43]:
userforGenre("Action")

IdUserSteam con más horas de Playtime: 76561198007282725
Total de horas de Playtime del usuario: 1635566


ReleaseYear,1993,1995,1996,1998,1999,2000,2001,2002,2003,2004,...,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017
IdUserSteam,,,,,,,,,,,,,,,,,,,,,
76561198007282725,0,217,0,0,44,70644,13,238,7673,127411,...,224,108326,78083,93757,378296,120306,130452,312238,29298,43327


#### Guardamos el dataframe para las consultas de la API ####

*Lo exportamos a parquet*.

In [44]:
genreTab = pa.Table.from_pandas(dfUxG)
dir2 = "datafunc/userbygenre.parquet"
pq.write_table(genreTab,dir2)

#### **Función bestDeveloper** ####
---

*La función **bestDEveloper** le permite al usuario **filtrar** por el **año** específico de lanzamiento de apps y mostrarle el **top 3 de los juegos más recomendados** para ese año.*

#### Creamos el dataframe para facilitar esta función ####

*Este dataframe solo contará con las columnas que nos interesan:*

1. *IdApp*
2. *TrueCount*
3. *Developer*
4. *ReleaseYear*

*Unimos el dataframe **dfRecommends** con el dataframe **dfGames**, en la columna IdApp*

1. *Con el fin de traer las columnas **Developer** y **ReleaseYear**.*
2. *Nos quedamos solo con las columnas que necesitaremos.*

In [51]:
dfBestDevs = pd.merge(dfRecommends,dfGames, on="IdApp", how="left")
dfBestDevs.drop(columns=["TotalReviews","RecommendsPercen", "Name", "Publisher", "Price", "ReleaseMonth"], inplace=True)
dfBestDevs.head(1)

,IdApp,TrueCount,Developer,ReleaseYear
0,10,56.0,Valve,2000


#### Creamos la función bestDeveloper ####

*Esta función recibirá como parametros: **Año** y el **dataframe**:*

1. *Año = Cadena de String del año de lanzamiento.*

In [52]:
def bestDeveloper(year:str, data = dfBestDevs):
    #Filtramos el dataframe y solo nos quedamos con el año que queremos.
    data = data[data["ReleaseYear"]==year]
    data = data.drop(columns="IdApp")
    
    #Agrupamos por usuario y calculamos la suma de recomendaciones por developer.
    groupByDev = data.groupby("Developer")['TrueCount'].sum().reset_index()
    
    #Ordenamos los resultados de mayor a menor y filtramos solo los 3 primeros
    bestdev = groupByDev.sort_values(by="TrueCount",ascending=False).head(3)
    
    #Reseatamos los indices y los convertimos en una columna de Ranking
    bestdev.reset_index(drop=True, inplace=True)
    bestdev.index += 1
    bestdev.reset_index(inplace=True)
    bestdev.rename(columns={'index': 'Puesto'}, inplace=True)
    
    #Eliminamos columna TrueCount
    bestdev.drop(columns="TrueCount", inplace=True)
    
    return bestdev

*Revisamos que funcione correctamente:*

In [47]:
bestDeveloper("2013")

,Puesto,Developer
0,1,Bohemia Interactive
1,2,Facepunch Studios
2,3,Digital Extremes


In [48]:
bestDeveloper("2017")

,Puesto,Developer
0,1,Smartly Dressed Games
1,2,Freejam
2,3,"Studio Wildcard,Instinct Games,Efecto Studios,..."


#### Guardamos el dataframe para las consultas de la API ####

*Lo exportamos a parquet*.

In [49]:
bestTab = pa.Table.from_pandas(dfBestDevs)
dir3 = "datafunc/bestDevs.parquet"
pq.write_table(bestTab,dir3)

#### **Función developerReviewsAnalysis** ####
---

*La función **developerReviewsAnalysis** le permite al usuario **filtrar** por un **desarrollador** específico y mostrarle la  **cantidad de reseñas categorizadas como positivas** y la **cantidad de reseñas categorizadas como negativas**.*

#### Creamos el dataframe para facilitar esta función ####

*Este dataframe solo contará con las columnas que nos interesan:*

1. *IdApp*
2. *SentAn1*
3. *Developer*

*Unimos el dataframe **dfFeelings** con el dataframe **dfGames**, en la columna IdApp*

1. *Con el fin de traer la columna **Developer**.*
2. *Nos quedamos solo con las columnas que necesitaremos.*

In [53]:
dfRevFeels = pd.merge(dfFeelings,dfGames, on="IdApp", how="left")
dfRevFeels = dfRevFeels[["IdApp", "Developer", "SenAn1"]]
dfRevFeels["Developer"] = dfRevFeels["Developer"].str.upper()
dfRevFeels.head(15)

,IdApp,Developer,SenAn1
0,10,VALVE,2
1,10,VALVE,2
2,10,VALVE,1
3,10,VALVE,2
4,10,VALVE,2
5,10,VALVE,1
6,10,VALVE,2
7,10,VALVE,1
8,10,VALVE,1
9,10,VALVE,2


#### Creamos la función developerReviewsAnalysis ####

*Esta función recibirá como parametros: **developer** y el **dataframe**:*

1. *Developer = Cadena de String del nombre del desarrollador.*

In [54]:
def developerReviewsAnalysis(developer:str, data = dfRevFeels):
    developer = developer.upper()
    #Filtramos el dataframe y solo nos quedamos con el developer que queremos.
    data = data[data["Developer"]==developer]
    data = data.drop(columns=["IdApp", "Developer"])
    
    #Transformamos la columna SenAn1 en columnas dummies
    dummy = pd.get_dummies(data['SenAn1'])
    dummy.drop(columns=1, inplace=True)
    dummy.rename(columns={0 : "Negative", 2 : "Positive"}, inplace=True)

    return dummy.sum()

*Revisamos que funcione correctamente:*

In [55]:
developerReviewsAnalysis("Valve")

Negative    1344
Positive    4989
dtype: int64

In [56]:
developerReviewsAnalysis("blue byte")

Negative    5
Positive    1
dtype: int64

#### Guardamos el dataframe para las consultas de la API ####

*Lo exportamos a parquet*.

In [269]:
revTab = pa.Table.from_pandas(dfRevFeels)
dir4 = "datafunc/devRevAna.parquet"
pq.write_table(revTab,dir4)